In [ ]:
import tensorflow as tf
import keras
from keras import layers
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
!pip install tensorflow-model-optimization
from tensorflow_model_optimization.python.core.sparsity.keras import prune, pruning_callbacks
import tensorflow_model_optimization as tfmot
from tensorflow_model_optimization.sparsity import keras as sparsity
from tensorflow.keras.models import load_model
from google.colab import files
import shutil
import gc
from keras.datasets import cifar10

np.random.seed(42)
tf.random.set_seed(42)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 2.3 MB/s eta 0:00:00


In [ ]:
def build_lenet_model(input_shape, num_classes):
    model = keras.Sequential([
        keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Flatten(),
        keras.layers.Dense(128, activation='relu'),
        keras.layers.Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Load and preprocess MNIST data
num_classes = 10
input_shape = (32, 32, 3)

# Load the data and split it between train and test sets
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)



170498071/170498071 [==============================] - 4s 0us/step
x_train shape: (50000, 32, 32, 3, 1)
50000 train samples
10000 test samples


In [ ]:
def train_lenet(model, train_data, train_labels, test_data, test_labels):
    model.save_weights(init_weights)
    model.save(init_model)
    initailsoftmax = model.predict(test_data)
    initaildf = pd.DataFrame(initailsoftmax)
    filename = filepath + '0_softmax.csv'
    initaildf.to_csv(filename,index=False)
    epoch = 1
    while epoch < 26:
        print(f"Epoch {epoch}:")

        # Training on one epoch
        model.fit(train_data, train_labels, epochs=1, batch_size=64, verbose=1)

        # Evaluate on the test dataset
        loss, accuracy = model.evaluate(test_data, test_labels, verbose=0)
        results = model.predict(test_data)
        softmax_df = pd.DataFrame(results)
        filename = filepath+str(epoch)+'_softmax.csv'
        softmax_df.to_csv(filename,index=False)

        print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")
        if epoch ==25:
          model.save_weights(final_weights)
          model.save(final_model)
        epoch += 1




In [ ]:
model_name = 'LeNet'
dataset = 'CIFAR10'

# Creating Base Models

In [ ]:
filepath = '/content/base/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset +'_init.tf'
final_weights = filepath + model_name + dataset +'_final_weights.tf'
final_model= filepath + model_name + dataset +'_final.tf'

In [ ]:
!mkdir '/content/base/'

In [ ]:
base = build_lenet_model(input_shape, num_classes)

In [ ]:
base.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 15, 15, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 6, 6, 64)          0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 2304)              0         
                                                                 
 dense (Dense)               (None, 128)               2

In [ ]:
train_lenet(base,x_train,y_train,x_test,y_test)

313/313 [==============================] - 3s 4ms/step
Epoch 1:
313/313 [==============================] - 1s 4ms/step
Test Loss: 1.2061899900436401, Test Accuracy: 0.5666999816894531
Epoch 2:
313/313 [==============================] - 1s 3ms/step
Test Loss: 1.0377649068832397, Test Accuracy: 0.6371999979019165
Epoch 3:
313/313 [==============================] - 1s 2ms/step
Test Loss: 0.9755279421806335, Test Accuracy: 0.6610000133514404
Epoch 4:
313/313 [==============================] - 1s 2ms/step
Test Loss: 0.9423867464065552, Test Accuracy: 0.6783000230789185
Epoch 5:
313/313 [==============================] - 1s 2ms/step
Test Loss: 0.9233331084251404, Test Accuracy: 0.6886000037193298
Epoch 6:
313/313 [==============================] - 1s 2ms/step
Test Loss: 0.928912878036499, Test Accuracy: 0.6886000037193298
Epoch 7:
313/313 [==============================] - 1s 2ms/step
Test Loss: 0.9393592476844788, Test Accuracy: 0.6905999779701233
Epoch 8:
313/313 [=========================

In [ ]:
del (base)
gc.collect()
keras.backend.clear_session()

# Independant model one

In [ ]:
filepath = '/content/independant_1/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + '_final.tf'

In [ ]:
!mkdir  '/content/independant_1/'

In [ ]:
model_independant_1 = build_lenet_model(input_shape, num_classes)

In [ ]:
train_lenet(model_independant_1,x_train,y_train,x_test,y_test)

313/313 [==============================] - 1s 2ms/step
Epoch 1:
313/313 [==============================] - 1s 2ms/step
Test Loss: 1.2111927270889282, Test Accuracy: 0.5756000280380249
Epoch 2:
313/313 [==============================] - 1s 3ms/step
Test Loss: 1.052960753440857, Test Accuracy: 0.6309000253677368
Epoch 3:
313/313 [==============================] - 1s 2ms/step
Test Loss: 0.9895525574684143, Test Accuracy: 0.6554999947547913
Epoch 4:
313/313 [==============================] - 1s 2ms/step
Test Loss: 0.9495759010314941, Test Accuracy: 0.6722999811172485
Epoch 5:
313/313 [==============================] - 1s 2ms/step
Test Loss: 0.9176976680755615, Test Accuracy: 0.6854000091552734
Epoch 6:
313/313 [==============================] - 1s 2ms/step
Test Loss: 0.9124781489372253, Test Accuracy: 0.6906999945640564
Epoch 7:
313/313 [==============================] - 1s 2ms/step
Test Loss: 0.9068896770477295, Test Accuracy: 0.697700023651123
Epoch 8:
313/313 [==========================

In [ ]:
del (model_independant_1)
gc.collect()
keras.backend.clear_session()

# Independant 2

In [ ]:
filepath = '/content/independant_2/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights =filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + '_final.tf'

In [ ]:
!mkdir '/content/independant_2/'

In [ ]:
model_independant_2 = build_lenet_model(input_shape, num_classes)

In [ ]:
train_lenet(model_independant_2,x_train,y_train,x_test,y_test)

313/313 [==============================] - 1s 2ms/step
Epoch 1:
313/313 [==============================] - 1s 2ms/step
Test Loss: 1.2225019931793213, Test Accuracy: 0.5684000253677368
Epoch 2:
313/313 [==============================] - 1s 2ms/step
Test Loss: 1.0676370859146118, Test Accuracy: 0.628600001335144
Epoch 3:
313/313 [==============================] - 1s 2ms/step
Test Loss: 0.9769660234451294, Test Accuracy: 0.6592000126838684
Epoch 4:
313/313 [==============================] - 1s 2ms/step
Test Loss: 0.9644829034805298, Test Accuracy: 0.6621999740600586
Epoch 5:
313/313 [==============================] - 1s 3ms/step
Test Loss: 0.967083215713501, Test Accuracy: 0.6704000234603882
Epoch 6:
313/313 [==============================] - 1s 2ms/step
Test Loss: 0.95204758644104, Test Accuracy: 0.6791999936103821
Epoch 7:
313/313 [==============================] - 1s 2ms/step
Test Loss: 0.9622297883033752, Test Accuracy: 0.6855999827384949
Epoch 8:
313/313 [============================

In [ ]:
del (model_independant_2)
gc.collect()
keras.backend.clear_session()

#Download Independant Models

In [ ]:
shutil.make_archive("base", 'zip', "base")
files.download("base.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
shutil.make_archive("independant_1", 'zip', "independant_1")
files.download("independant_1.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:

shutil.make_archive("independant_2", 'zip', "independant_2")
files.download("independant_2.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

---

# Distillation

In [ ]:
def train_custom(model, train_data, train_labels, test_data, test_labels):
    initailsoftmax = model.student.predict(test_data)
    model.student.save_weights(init_weights)
    model.student.save(init_model)
    initaildf = pd.DataFrame(initailsoftmax)
    filename = filepath + '0_softmax.csv'
    initaildf.to_csv(filename,index=False)
    epoch = 1
    while epoch < 26:
        print(f"Epoch {epoch}:")

        # Training on one epoch
        model.fit(train_data, train_labels, epochs=1, batch_size=64, verbose=1)

        # Evaluate on the test dataset
        results = model.student.predict(test_data)
        softmax_df = pd.DataFrame(results)
        filename = filepath+str(epoch)+'_softmax.csv'
        softmax_df.to_csv(filename,index=False)
        loss,accuracy = model.student.evaluate(test_data,test_labels)
        print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

        if epoch ==25:
          model.student.save_weights(final_weights)
          model.student.save(final_model)
        epoch += 1

# TO DO EVALUATE FUNCTION


In [ ]:
class Distiller(tf.keras.Model):
    def __init__(self, student, teacher):
        super().__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha,
        temperature,
    ):
        super().compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)

            distillation_loss = (
                self.distillation_loss_fn(
                    tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                    tf.nn.softmax(student_predictions / self.temperature, axis=1),
                )
                * self.temperature**2
            )

            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )

        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)
        print(y_prediction)
        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results

# Student 0.1 Alpha

In [ ]:
model_path = '/content/base/LeNetCIFAR10_init.tf'
student = load_model(model_path)
student_scratch = tf.keras.models.clone_model(student)


In [ ]:
teacher_model_path = '/content/base/LeNetCIFAR10_final.tf'
techer_model = load_model(teacher_model_path)

In [ ]:
distiller = Distiller(student=student, teacher=techer_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    metrics=[tf.keras.metrics.CategoricalCrossentropy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=3,
)

In [ ]:
filepath = '/content/student_same/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + 'final.tf'

In [ ]:
!mkdir '/content/student_same/'

In [ ]:
train_custom(distiller, x_train,y_train,x_test,y_test)

313/313 [==============================] - 1s 3ms/step
Epoch 1:


/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5575: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


313/313 [==============================] - 1s 3ms/step - loss: 1.3891 - accuracy: 0.5101
Test Loss: 1.389121413230896, Test Accuracy: 0.5101000070571899
Epoch 2:
313/313 [==============================] - 1s 3ms/step - loss: 1.2479 - accuracy: 0.5629
Test Loss: 1.2479034662246704, Test Accuracy: 0.5629000067710876
Epoch 3:
313/313 [==============================] - 1s 3ms/step - loss: 1.1472 - accuracy: 0.5979
Test Loss: 1.1472395658493042, Test Accuracy: 0.5978999733924866
Epoch 4:
313/313 [==============================] - 1s 3ms/step - loss: 1.0791 - accuracy: 0.6224
Test Loss: 1.0791150331497192, Test Accuracy: 0.6223999857902527
Epoch 5:
313/313 [==============================] - 1s 3ms/step - loss: 1.0322 - accuracy: 0.6417
Test Loss: 1.032204508781433, Test Accuracy: 0.641700029373169
Epoch 6:
313/313 [==============================] - 1s 4ms/step - loss: 1.0025 - accuracy: 0.6547
Test Loss: 1.0024852752685547, Test Accuracy: 0.654699981212616
Epoch 7:
313/313 [=================

In [ ]:
del (student)
gc.collect()
keras.backend.clear_session()

# Student same Alpha 0.5

In [ ]:
model_path = '/content/base/LeNetCIFAR10_init.tf'
student = load_model(model_path)
student_scratch = tf.keras.models.clone_model(student)

In [ ]:
distiller = Distiller(student=student, teacher=techer_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    metrics=[tf.keras.metrics.CategoricalCrossentropy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.5,
    temperature=3,
)

In [ ]:
filepath = '/content/student_same_50/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + 'final.tf'

In [ ]:
!mkdir '/content/student_same_50/'

In [ ]:
train_custom(distiller, x_train,y_train,x_test,y_test)

313/313 [==============================] - 1s 2ms/step
Epoch 1:


/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5575: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


313/313 [==============================] - 1s 3ms/step - loss: 1.3944 - accuracy: 0.5098
Test Loss: 1.3944451808929443, Test Accuracy: 0.5098000168800354
Epoch 2:
313/313 [==============================] - 1s 3ms/step - loss: 1.2518 - accuracy: 0.5601
Test Loss: 1.2518205642700195, Test Accuracy: 0.5601000189781189
Epoch 3:
313/313 [==============================] - 1s 3ms/step - loss: 1.1485 - accuracy: 0.5997
Test Loss: 1.1484973430633545, Test Accuracy: 0.5996999740600586
Epoch 4:
313/313 [==============================] - 1s 3ms/step - loss: 1.0834 - accuracy: 0.6222
Test Loss: 1.0833909511566162, Test Accuracy: 0.6222000122070312
Epoch 5:
313/313 [==============================] - 1s 4ms/step - loss: 1.0334 - accuracy: 0.6398
Test Loss: 1.033415675163269, Test Accuracy: 0.6398000121116638
Epoch 6:
313/313 [==============================] - 1s 3ms/step - loss: 1.0033 - accuracy: 0.6526
Test Loss: 1.0033020973205566, Test Accuracy: 0.6525999903678894
Epoch 7:
313/313 [==============

In [ ]:
del (student)
gc.collect()
keras.backend.clear_session()

# Student same Alpha 0.9

In [ ]:
model_path = '/content/base/LeNetCIFAR10_init.tf'
student = load_model(model_path)
student_scratch = tf.keras.models.clone_model(student)

distiller = Distiller(student=student, teacher=techer_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    metrics=[tf.keras.metrics.CategoricalCrossentropy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.5,
    temperature=3,
)

filepath = '/content/student_same_90/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + 'final.tf'

!mkdir '/content/student_same_90/'

In [ ]:
train_custom(distiller, x_train,y_train,x_test,y_test)

313/313 [==============================] - 1s 2ms/step
Epoch 1:


/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5575: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


313/313 [==============================] - 1s 3ms/step - loss: 1.3835 - accuracy: 0.5133
Test Loss: 1.3835070133209229, Test Accuracy: 0.5133000016212463
Epoch 2:
313/313 [==============================] - 1s 4ms/step - loss: 1.2456 - accuracy: 0.5620
Test Loss: 1.245585560798645, Test Accuracy: 0.5619999766349792
Epoch 3:
313/313 [==============================] - 1s 3ms/step - loss: 1.1500 - accuracy: 0.5960
Test Loss: 1.1500041484832764, Test Accuracy: 0.5960000157356262
Epoch 4:
313/313 [==============================] - 1s 3ms/step - loss: 1.0859 - accuracy: 0.6221
Test Loss: 1.0859287977218628, Test Accuracy: 0.6220999956130981
Epoch 5:
313/313 [==============================] - 1s 3ms/step - loss: 1.0422 - accuracy: 0.6394
Test Loss: 1.0421785116195679, Test Accuracy: 0.6394000053405762
Epoch 6:
313/313 [==============================] - 1s 4ms/step - loss: 1.0145 - accuracy: 0.6479
Test Loss: 1.0144973993301392, Test Accuracy: 0.6478999853134155
Epoch 7:
313/313 [==============

In [ ]:
del (student)
gc.collect()
keras.backend.clear_session()

---

# Download Distill Data

In [ ]:
shutil.make_archive("student_same", 'zip', "student_same")
files.download("student_same.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
shutil.make_archive("student_same_50", 'zip', "student_same_50")
files.download("student_same_50.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
shutil.make_archive("student_same_90", 'zip', "student_same_90")
files.download("student_same_90.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

---

# Qunatisation

https://github.com/sonalimedani/TF_Quantization/blob/master/quantization.ipynb

https://medium.com/@sonalimedani/post-training-quantization-with-tensorflow-lite-on-a-keras-model-f373068966c4

**Default**: 8 Bit

In [ ]:
path = '/content/base/LeNetCIFAR10_final.tf'
model = tf.keras.models.load_model(path)
model.evaluate(x_test,y_test)

313/313 [==============================] - 1s 4ms/step - loss: 2.4273 - accuracy: 0.6339


[2.427269220352173, 0.633899986743927]

In [ ]:
filepath = '/content/quantised_8_bit/'
!mkdir '/content/quantised_8_bit/'

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()
#saving converted model in "converted_quant_model.tflite" file
open(filepath+"8_bit.tflite", "wb").write(tflite_quant_model)

323328

In [ ]:
import os
print("Float model in Mb:", os.path.getsize(filepath+"8_bit.tflite") / float(2**20))
print("Quantized model in Mb:", os.path.getsize(filepath+"8_bit.tflite") / float(2**20))
print("Compression ratio:", os.path.getsize(filepath+"8_bit.tflite")/os.path.getsize(path))

Float model in Mb: 0.308349609375
Quantized model in Mb: 0.308349609375
Compression ratio: 78.9375


In [ ]:
interpreter = tf.lite.Interpreter(model_path=(filepath+"8_bit.tflite"))
interpreter.allocate_tensors()
# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
# Test model on some input data.
input_shape = input_details[0]['shape']
acc=0
results_default = pd.DataFrame()
for i in range(len(x_test)):
    input_data = x_test[i].reshape(input_shape)
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    row = pd.DataFrame(output_data)
    results_default = pd.concat([results_default, row], ignore_index=True)
    if(np.argmax(output_data) == np.argmax(y_test[i])):
        acc+=1
acc = acc/len(x_test)
print(acc*100)

63.36000000000001


In [ ]:
results_default

,0,1,2,3,4,5,6,7,8,9
0,3.085574e-04,2.101067e-09,9.365412e-08,9.576411e-01,1.881767e-11,3.284454e-02,7.853651e-03,1.033397e-08,1.311976e-03,4.018681e-05
1,7.030472e-10,1.945706e-04,2.226929e-19,5.752910e-17,2.890692e-18,2.004448e-26,1.113514e-25,4.307149e-26,9.998023e-01,3.185183e-06
2,5.464497e-05,1.780022e-01,3.207399e-06,1.134617e-06,5.127932e-06,6.653705e-08,3.511103e-11,8.913897e-04,7.319981e-01,8.904407e-02
3,9.969416e-01,1.064995e-10,7.461209e-05,6.793064e-06,1.214356e-03,7.811304e-09,1.378640e-06,3.796962e-09,1.761185e-03,1.035231e-07
4,7.739738e-20,5.147049e-13,3.204761e-07,3.521553e-06,9.999759e-01,1.111178e-09,2.015488e-05,6.049402e-12,6.303562e-11,1.335817e-13
...,...,...,...,...,...,...,...,...,...,...
9995,9.643251e-08,2.140988e-18,1.842671e-07,9.839637e-01,3.612719e-07,1.232421e-02,1.284019e-11,1.022754e-15,3.711489e-03,2.212922e-11
9996,1.014946e-17,2.180808e-21,1.854535e-06,5.024426e-02,2.977131e-05,9.495816e-01,1.425257e-04,4.522447e-08,4.386495e-18,1.431518e-15
9997,1.018589e-19,9.514478e-23,4.950550e-07,5.488214e-04,1.334043e-13,9.994505e-01,6.025249e-16,2.368125e-07,4.132414e-21,5.449133e-16
9998,7.043039e-10,5.315771e-02,1.531296e-11,2.637366e-03,1.532932e-02,9.288756e-01,2.355547e-11,2.331842e-11,3.313002e-11,8.304712e-11


In [ ]:
results_default.to_csv((filepath+"25_softmax.csv"),index=False)

**16 bit**

In [ ]:
model = tf.keras.models.load_model(path)


In [ ]:
filepath = '/content/quantised_16_bit/'
!mkdir '/content/quantised_16_bit/'

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
tflite_quant_model = converter.convert()
#saving converted model in "converted_quant_model.tflite" file
open((filepath +"16_bit.tflite"), "wb").write(tflite_quant_model)

635984

In [ ]:
import os
print("Float model in Mb:", os.path.getsize(filepath +"16_bit.tflite") / float(2**20))
print("Quantized model in Mb:", os.path.getsize(filepath +"16_bit.tflite") / float(2**20))
print("Compression ratio:", os.path.getsize(filepath +"16_bit.tflite")/os.path.getsize(path))

Float model in Mb: 0.6065216064453125
Quantized model in Mb: 0.6065216064453125
Compression ratio: 155.26953125


In [ ]:
interpreter = tf.lite.Interpreter(model_path=filepath +"16_bit.tflite")
interpreter.allocate_tensors()
# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
# Test model on some input data.
input_shape = input_details[0]['shape']
acc=0
results_16_b = pd.DataFrame()
for i in range(len(x_test)):
    input_data = x_test[i].reshape(input_shape)
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    row = pd.DataFrame(output_data)
    results_16_b = pd.concat([results_16_b, row], ignore_index=True)
    if(np.argmax(output_data) == np.argmax(y_test[i])):
        acc+=1
acc = acc/len(x_test)
print(acc*100)

63.4


In [ ]:
results_16_b

,0,1,2,3,4,5,6,7,8,9
0,2.641963e-04,1.676827e-09,9.493932e-08,9.453492e-01,2.866321e-11,4.389160e-02,9.123744e-03,1.138396e-08,1.335063e-03,3.616158e-05
1,6.872629e-10,2.023901e-04,2.601279e-19,5.378424e-17,2.829087e-18,2.557768e-26,1.072491e-25,4.690962e-26,9.997944e-01,3.188844e-06
2,6.326695e-05,1.818717e-01,3.306275e-06,1.077952e-06,5.167411e-06,6.869912e-08,4.382904e-11,9.502213e-04,7.279609e-01,8.914422e-02
3,9.964735e-01,1.184853e-10,9.933043e-05,7.442326e-06,1.330521e-03,8.329307e-09,1.316051e-06,3.305948e-09,2.087746e-03,1.176326e-07
4,9.673276e-20,4.961078e-13,4.347058e-07,4.137374e-06,9.999719e-01,1.360211e-09,2.353691e-05,6.456095e-12,7.533849e-11,1.357388e-13
...,...,...,...,...,...,...,...,...,...,...
9995,1.224353e-07,2.435328e-18,3.141976e-07,9.819083e-01,4.565735e-07,1.528018e-02,1.496418e-11,1.695925e-15,2.810597e-03,2.310159e-11
9996,8.840260e-18,1.663779e-21,1.555168e-06,3.943571e-02,2.574912e-05,9.604260e-01,1.110284e-04,3.448152e-08,4.122377e-18,1.065039e-15
9997,6.839145e-20,7.424943e-23,4.395152e-07,4.716293e-04,1.352721e-13,9.995277e-01,4.520719e-16,1.973148e-07,3.120693e-21,2.981125e-16
9998,6.577368e-10,6.030961e-02,1.838099e-11,2.614410e-03,2.184755e-02,9.152284e-01,2.560421e-11,2.560450e-11,3.841354e-11,7.748891e-11


In [ ]:
results_16_b.to_csv(filepath +'25_softmax.csv',index =False)

# Download Data

In [ ]:
shutil.make_archive("quantised_16_bit", 'zip', "quantised_16_bit")
files.download("quantised_16_bit.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
shutil.make_archive("quantised_8_bit", 'zip', "quantised_8_bit")
files.download("quantised_8_bit.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>